<a href="https://colab.research.google.com/github/MatteoRobbiati/notebooks/blob/main/QTI-QML-tutorial/QuantumClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qibo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.8/225.8 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 KB 12.3 MB/s eta 0:00:00


In [4]:
import qibo
from qibo import gates, hamiltonians, derivative
from qibo.models import Circuit

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid', font_scale=1.5)

### Evaluating analytical gradients on a quantum circuit

In [10]:
nqubits = 1
layers = 2

c = Circuit(nqubits)
for l in range(layers):
  c.add(gates.RY(q=0, theta=0))
  c.add(gates.RY(q=0, theta=0))
  c.add(gates.RZ(q=0, theta=0))
  c.add(gates.RZ(q=0, theta=0))
c.add(gates.M(0))


MeasurementResult(qubits=(0,), nshots=0)

In [11]:
print(c.draw())
print(c.summary())

q0: ─RY─RY─RZ─RZ─RY─RY─RZ─RZ─M─
Circuit depth = 9
Total number of gates = 9
Number of qubits = 1
Most common gates:
ry: 4
rz: 4
measure: 1


In [12]:
def inject_parameters(circuit, parameters, x):
  params = []
  index = 0
  
  for l in range(layers):
    # embed the first feature
    params.append(parameters[index] * x[0])
    params.append(parameters[index + 1])
    # embed the second feature
    params.append(parameters[index + 2] * x[1])
    params.append(parameters[index + 3])
    index += 4

  circuit.set_parameters(params)
  return circuit

In [26]:
nparams = len(c.get_parameters())
initial_parameters = np.random.randn(nparams) * 5

# set them into the circuit together with an x
x = [0.5, 0.1]
c = inject_parameters(c, initial_parameters, x)

In [24]:
# define an hamiltonian
h = hamiltonians.Z(nqubits)

# which can be used passing a quantum state 
dummy_state = np.ones(2 ** nqubits) / np.sqrt(2 ** nqubits)

h.expectation(dummy_state)

<tf.Tensor: shape=(), dtype=float64, numpy=0.0>

In [29]:
h.expectation(c.execute(nshots=1000).state())

<tf.Tensor: shape=(), dtype=float64, numpy=0.40862942703082394>

<tf.Tensor: shape=(2,), dtype=complex128, numpy=array([ 0.13996064+0.52544867j, -0.72944942+0.41499188j])>